In [7]:
import os
import nest_asyncio
import asyncio
from typing import Optional

from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(
        base_url='https://api.deepseek.com',
        model='deepseek-chat',
        temperature=0.7,
        api_key=os.getenv('DEEPSEEK_SK'),
    )
    print('初始化大模型成功')

except Exception as e:
    print(f'初始化大模型失败: {e}')
    llm = None
    
    
    
# 汇总链
summarize_chain: Runnable = ChatPromptTemplate.from_messages([
    ('system', '请简明扼要的总结以下主题'),
    ('user', '{topic}')
]) | llm | StrOutputParser()

# 问题链
question_chain: Runnable = ChatPromptTemplate.from_messages([
    ('system', '请对以下主题生成3个有趣的问题'),
    ('user', '{topic}')
]) | llm | StrOutputParser()

terms_chain: Runnable = ChatPromptTemplate.from_messages([
    ("system", "请从以下主题中提取 5‑10 个关键词，用逗号分隔"),
    ('user', '{topic}')
]) | llm | StrOutputParser()

# 构建并行链
map_chain = RunnableParallel({
    'summary': summarize_chain,
    'question': question_chain,
    'key_terms': terms_chain,
    'topic': RunnablePassthrough(),
})

# 汇总
synthesis_chain: Runnable = ChatPromptTemplate.from_messages([
    ('system', """
        请根据以下信息,
        摘要: {summary},
        问题: {question},
        关键词: {key_terms},
        生成综合答案.
    """),
    ('user', '原始主题: {topic}')
])

full_parallel_chain = map_chain | synthesis_chain | llm | StrOutputParser()

# result = full_parallel_chain.invoke('太空探索的历史')
# print(result)

async def run_parallel_example(topic: str) -> None: 
    if not llm:
        print('LLM未初始化')
        return

    print(f'--- 运行示例: {topic} ---')
    
    try:
        response = await full_parallel_chain.ainvoke(topic)
        print('----- 最终响应 -----')
        print(response)
    except Exception as e:
        print(f'执行出错: {e}')
        

if __name__ == '__main__':
    # nest_asyncio.apply()
    loop = asyncio.get_running_loop()
    asyncio.run(run_parallel_example('太空探索的历史'))
    



初始化大模型成功
--- 运行示例: 太空探索的历史 ---
----- 最终响应 -----
太空探索的历史是一部充满挑战与突破的壮丽史诗，它跨越了政治竞争、科学发现与技术进步的交织发展。以下是对这一历程的综合梳理：

### 🌍 **早期探索（1950s-1960s）**
- **人造卫星里程碑**：1957年，苏联发射 **“斯普特尼克1号”** ，人类首次进入太空时代，引发全球震动。
- **载人航天突破**：1961年，苏联宇航员加加林乘坐 **“东方1号”** 完成首次绕地球飞行，而美国随后通过“水星计划”紧追不舍。
- **技术竞争**：这一时期以火箭技术（如苏联R-7与美国“土星5号”）为核心，奠定了轨道飞行的基础。

### 🌕 **登月竞赛（1960s-1970s）**
- **阿波罗计划巅峰**：1969年，美国 **“阿波罗11号”** 成功登陆月球，阿姆斯特朗的“个人一小步”成为人类探索精神的象征。
- **科学收获**：通过带回月岩样本与部署实验设备，人类首次实地研究地外天体，重塑了对月球形成与太阳系演化的认知。

### 🛰️ **空间站时代（1970s至今）**
- **早期实验站**：苏联“礼炮号”与美国“天空实验室”验证了长期太空居住的可行性。
- **国际合作典范**：**国际空间站（ISS）** 自1998年起成为多国协作的平台，支持微重力研究、地球观测与深空任务准备。

### 🚀 **深空探测拓展**
- **行星探索**：从“旅行者号”飞掠外行星，到“好奇号”火星车分析火星地质，探测器揭示了太阳系的多样性。
- **寻找生命迹象**：任务如“毅力号”在火星采集样本，试图解答地球外生命存在的可能性。

### 🌌 **商业航天崛起（21世纪）**
- **可重复使用技术**：SpaceX的“猎鹰9号”火箭与“龙飞船”大幅降低发射成本，推动太空产业化。
- **新兴领域**：蓝色起源、维珍银河等企业开启太空旅游，而“星链”等项目正重塑全球通信网络。

### 🔭 **未来展望**
- **重返月球**：NASA的“阿尔忒弥斯计划”目标在2025年后建立可持续月球基地。
- **火星任务**：多国计划在2030年代实现载人火星探索，挑战生命支持与长期航行技术。
- **深空前沿**：望远镜如“詹姆斯·韦伯”探测系外行星，为人类